In [1]:
import keras
import tensorflow
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

2025-05-25 09:22:15.381674: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-25 09:22:15.482646: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-25 09:22:15.565430: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748144235.637000    6846 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748144235.656481    6846 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748144235.819892    6846 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
# Data Wine Data

data = pd.read_csv(
  'https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv',
  sep=';'
)

data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [3]:
# Train Test

train,test = train_test_split(data, test_size=0.25)
train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1632,7.1,0.280,0.49,6.50,0.041,28.0,111.0,0.99260,3.41,0.58,12.2,8
1827,7.3,0.190,0.24,6.30,0.054,34.0,231.0,0.99640,3.36,0.54,10.0,6
4152,6.2,0.200,0.25,15.00,0.055,8.0,120.0,0.99767,3.19,0.53,9.6,6
3932,6.9,0.220,0.28,7.80,0.050,43.0,116.0,0.99326,3.22,0.60,11.5,8
3321,6.0,0.260,0.33,4.35,0.040,15.0,80.0,0.98934,3.29,0.50,12.7,6
...,...,...,...,...,...,...,...,...,...,...,...,...
3727,5.7,0.265,0.28,6.90,0.036,46.0,150.0,0.99299,3.36,0.44,10.8,7
4169,6.7,0.210,0.34,1.50,0.035,45.0,123.0,0.98949,3.24,0.36,12.6,7
4133,5.7,0.250,0.27,11.50,0.040,24.0,120.0,0.99411,3.33,0.31,10.8,6
382,7.5,0.350,0.28,9.60,0.051,26.0,157.0,0.99690,3.12,0.53,9.2,6


In [4]:
train_x = train.drop(['quality'],axis=1).values
train_y = train[['quality']].values.ravel()

# Test Data
test_x = train.drop(['quality'],axis=1).values
test_y = train[['quality']].values.ravel()

# Validation after splitting the Training Data
train_x,valid_x,train_y,valid_y = train_test_split(train_x,train_y,test_size=0.20,random_state=42)

signature = infer_signature(train_x,train_y)

In [5]:
# ANN Model

import mlflow.tensorflow


def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):
  
	# Noramlization 
	mean = np.mean(train_x,axis=0) # Mean of each col
	var = np.var(train_x,axis=0) # Var of Each col

	model = keras.Sequential(
		[
			keras.Input([train_x.shape[1]]),
			keras.layers.Normalization(mean=mean,variance=var),
			keras.layers.Dense(64,activation='relu'),
			keras.layers.Dense(1) # Classification
		]
	)

	# Model Compile
	model.compile(optimizer=keras.optimizers.SGD(
		learning_rate=params['lr'],momentum=params["momentum"]
	),
	loss="mean_squared_error",
	metrics=[keras.metrics.RootMeanSquaredError()])

	# Train and Track the Hyperparam with MLFlow tracking

	with mlflow.start_run(nested=True): # As we are trying with multiple combination, nested = True
		model.fit(train_x,train_y,validation_data=(valid_x,valid_y),
						epochs=epochs,
						batch_size=64)
		
		# Evaluate the model
		eval_result = model.evaluate(valid_x, valid_y, batch_size=64)

		eval_rmse = eval_result[1]

		# Log the params
		mlflow.log_param("learning_rate",params)
		mlflow.log_metric("Eval Rms", eval_rmse)

		# Log the model
		mlflow.tensorflow.log_model(
			model, 
			"model",
			signature=signature
		)

		return {
			'loss': eval_rmse,
			'status': STATUS_OK,
			'model': model
		}

	

In [6]:
# Objective Function for Hyperopt

def objective(params):

  # MlFlow will track the params and results for each run

  result = train_model(
    params, 
    epochs=3, 
    train_x=train_x, 
    train_y=train_y, 
    valid_x=valid_x, 
    valid_y=valid_y, 
    test_x=test_x, 
    test_y=test_y
  )

  return result

In [7]:
# Set all the parameters

space = {
 'lr': hp.loguniform('lr',np.log(1e-5),np.log(1e-1)),
 'momentum': hp.uniform("momentum",0.0,1.0)
}

In [10]:
# Set Exp

import mlflow.tensorflow


mlflow.set_experiment("/wine-quality")

# Create another run so that the nested run will work
with mlflow.start_run():

  # Conduct Hyperparameter search using Hyperopt
  trails = Trials()
  best = fmin(
    fn=objective,
    space=space,
    algo=tpe.suggest,
    max_evals=4,
    trials=trails
  )

  # Fetch the details of the best run
  best_run = sorted(trails.results, key=lambda x:x['loss'])[0]

  print(trails.results)

  # Log the best parameters, loss and model
  for key, value in best.items():
    mlflow.log_param(key, value)
    
  mlflow.log_metric("Eval_RMSE", best_run['loss'])
  mlflow.tensorflow.log_model(
    best_run['model'],
    "model",
    signature=signature,
  )

  # Print out the best params and loss
  print(f"Best Param: {best}")
  print(f"Best Eval EMSE: {best_run['loss']}")

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

Epoch 1/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 12s 273ms/step - loss: 39.7355 - root_mean_squared_error: 6.3036
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 14.5992 - root_mean_squared_error: 3.6582 - val_loss: 1.4611 - val_root_mean_squared_error: 1.2087

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 1.0166 - root_mean_squared_error: 1.0083
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1347 - root_mean_squared_error: 1.0648 - val_loss: 1.0118 - val_root_mean_squared_error: 1.0059

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 1.1350 - root_mean_squared_error: 1.0654
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.9394 - root_mean_squared_error: 0.9683 - val_loss: 0.7902 - val_root_mean_squared_error: 0.8890

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.6939 - root_mean_squared_error: 0.8330
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/ste

In [ ]:
# Inferencing Model

import mlflow

model_uri = 'runs:/d6afa02fdf47443bb97a85e0068fd121/model'

loaded_model = mlflow.pyfunc.load_model(model_uri)
predictions = loaded_model.predict(test_x)
predictions

115/115 ━━━━━━━━━━━━━━━━━━━━ 0s 546us/step


array([[5.2149525],
       [4.7655883],
       [4.4918528],
       ...,
       [3.6667314],
       [3.1362183],
       [3.8509052]], dtype=float32)